# Image Rotations: STL-10 + PyTorch

referring to the research paper _Unsupervised Representation Learning by Predicting Image Rotations_ by Spyros Gidaris et al.

In [ ]:
# Specify GPU to be used-
# %env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES = 0

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from tqdm import trange
import matplotlib.pyplot as plt
import numpy as np
import os, pickle, imageio

/home/majumdar/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/majumdar/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/majumdar/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
print(f"torch version: {torch.__version__}")

# Check if there are multiple devices (i.e., GPU cards)-
print(f"Number of GPU(s) available = {torch.cuda.device_count()}")

# Which GPU Is The Current GPU?
# print(f"current GPU: {torch.cuda.current_device()}")

# Get the name of the current GPU-
# print(f"Current GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

# Is PyTorch using a GPU?
# print(f"Is PyTorch using a GPU? {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"Current GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("PyTorch does not have access to GPU")

# print(torch.cuda.memory_summary())
# Device configuration-
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Available device is {device}')

torch version: 1.13.0+cu117
Number of GPU(s) available = 4
Current GPU: 0
Current GPU name: NVIDIA GeForce GTX TITAN X
Available device is cuda


In [3]:
# Specify training hyper-parameters-
batch_size = 64
num_epochs = 100

In [4]:
# Define transformations using albumentations-
transform_train = A.Compose(
    [
        # A.Resize(width = 32, height = 32),
        # A.RandomCrop(width = 20, height = 20),
        # A.Rotate(limit = 40, p = 0.9, border_mode = cv2.BORDER_CONSTANT),
        # A.HorizontalFlip(p = 0.5),
        # A.VerticalFlip(p = 0.1),
        A.RGBShift(r_shift_limit = 25, g_shift_limit = 25, b_shift_limit = 25, p = 0.9),
        A.OneOf([
            A.Blur(blur_limit = 3, p = 0.5),
            A.ColorJitter(p = 0.5),
        ], p = 1.0),
        A.Normalize(
            # mean = [0.4914, 0.4822, 0.4465],
            # std = [0.247, 0.243, 0.261],
            mean = [0, 0, 0],
            std = [1, 1, 1],
            max_pixel_value = 255,
        ),
        # This is not dividing by 255, which it does in PyTorch-
        ToTensorV2(),
    ]
)

transform_val = A.Compose(
    [
        A.Normalize(
            mean = [0, 0, 0],
            std = [1, 1, 1],
            max_pixel_value = 255
        ),
        ToTensorV2()
    ]
)


In [5]:
class STL10Dataset(Dataset):
    '''
    Custom STL-10 dataset only for unlabeled data!
    '''
    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir,
        self.transform = transform
    
    def __len__(self):
        totalFiles = 0
        totalDir = 0
        for base, dirs, files in os.walk(root_dir):
            # print(f'Searching in : {base}')
            for directories in dirs:
                totalDir += 1
            for Files in files:
                totalFiles += 1
        
        return totalFiles
    
    def __getitem__(self, idx):
        idx = idx + 1
        image = imageio.v3.imread(self.root_dir[0] + "unlabeled_image_png_" + str(idx) + ".png")
        
        if self.transform is not None:
            transformed = self.transform(image = image)
            image = transformed["image"]
            
            # Randomly select 0, 1, 2 or 3 for image rotation-
            ang = np.random.randint(low = 0, high = 4, size = None)
            image = TF.rotate(img = image, angle = ang * 90)
            
        # return image, label
        return image, ang

        

In [6]:
# Specify STL-10 unlabeled data directory-
root_dir = "/home/majumdar/Downloads/.data/stl10_binary/unlabeled/"

In [7]:
# Create training data-
unlabeled_data = STL10Dataset(
    root_dir = root_dir, transform = transform_train
)

In [8]:
# Create data loader-
unlabeled_loader = torch.utils.data.DataLoader(
    dataset = unlabeled_data, batch_size = batch_size,
    shuffle = True
)

In [ ]:
"""
# Sanity check-
img, y = next(iter(unlabeled_loader))

img.shape, y.shape
# img.shape, y.shape

# Unnormalize images-
img = img / 2 + 0.5

# Transpose axes for visualization-
img = np.transpose(img.numpy(), (0, 2, 3, 1))

img.shape
# (128, 96, 96, 3)
"""

In [12]:
# Visualize 50 images from training set-
plt.figure(figsize = (10, 9))

for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.imshow(img[i])
    # get current axes-
    ax = plt.gca()

    # hide x-axis-
    ax.get_xaxis().set_visible(False)

    # hide y-axis-
    ax.get_yaxis().set_visible(False)
    
plt.suptitle("Sample STL-10 unlabeled images")
plt.show()

In [ ]:
# del img, y

In [9]:
class ResNet_Block(nn.Module):
   
    def __init__(
        self, num_inp_channels,
        num_channels, num_bottleneck_channels,
        stride = 1, dropout = 0.2,
        use_1x1_conv = False, first_resblock = False
    ):
        super(ResNet_Block, self).__init__()
        
        self.num_inp_channels = num_inp_channels
        self.num_channels = num_channels
        self.num_bottleneck_channels = num_bottleneck_channels
        self.stride = stride
        self.dropout = dropout
        self.use_1x1_conv = use_1x1_conv
        self.first_resblock = first_resblock
    
        if self.first_resblock:
            self.conv1 = nn.Conv2d(
                in_channels = self.num_inp_channels, out_channels = self.num_bottleneck_channels,
                kernel_size = 1, padding = 0,
                stride = self.stride, bias = False
            )
            self.bn1 = nn.BatchNorm2d(num_features = self.num_bottleneck_channels)
        else:
            self.conv1 = nn.Conv2d(
                in_channels = self.num_inp_channels, out_channels = self.num_bottleneck_channels,
                kernel_size = 1, padding = 0,
                stride = self.stride, bias = False
            )
            self.bn1 = nn.BatchNorm2d(num_features = self.num_bottleneck_channels)
        
        self.conv2 = nn.Conv2d(
            in_channels = self.num_bottleneck_channels, out_channels = self.num_bottleneck_channels,
            kernel_size = 3, padding = 1,
            stride = 1, bias = False
        )
        self.bn2 = nn.BatchNorm2d(num_features = self.num_bottleneck_channels)
        self.dropout = nn.Dropout(p = self.dropout)
        
        self.conv3 = nn.Conv2d(
            in_channels = self.num_bottleneck_channels, out_channels = self.num_channels,
            kernel_size = 1, padding = 0,
            stride = 1, bias = False
        )
        self.bn3 = nn.BatchNorm2d(num_features = self.num_channels)
        
        if self.use_1x1_conv:
            self.conv4 = nn.Conv2d(
            in_channels = self.num_inp_channels, out_channels = self.num_channels,
            kernel_size = 1, padding = 0,
            stride = self.stride, bias = False
            )
            self.bn4 = nn.BatchNorm2d(num_features = self.num_channels)
        
        self.initialize_weights()

            
    def initialize_weights(self):
        for m in self.modules():
            # print(m)
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)

                '''
                # Do not initialize bias (due to batchnorm)-
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                '''
            
            elif isinstance(m, nn.BatchNorm2d):
                # Standard initialization for batch normalization-
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
    
    
    def forward(self, x):
        y = F.leaky_relu(self.bn1(self.conv1(x)))
        y = self.dropout(F.leaky_relu(self.bn2(self.conv2(y))))
        y = self.dropout(F.leaky_relu(self.bn3(self.conv3(y))))
        
        if self.use_1x1_conv:
            x = self.bn4(self.conv4(x))
            
        y += x
        # return F.relu(self.dropout(y))
        return F.leaky_relu(y)
    
    
    def shape_computation(self, x):
        print(f"Input shape: {x.shape}")
        y = self.bn1(self.conv1(x))
        print(f"First conv layer output shape: {y.shape}")
        y = self.bn2(self.conv2(y))
        print(f"Second conv layer output shape: {y.shape}")
        y = self.bn3(self.conv3(y))
        print(f"Third conv layer output shape: {y.shape}")
        
        if self.use_1x1_conv:
            x = self.bn4(self.conv4(x))
            print(f"Downsample with S = 2; identity connection output shape: {x.shape}")
            
        y += x
        print(f"Residual block output shape: {y.shape}")
        return None
    

In [10]:
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        
        self.conv1 = nn.Conv2d(
            in_channels = 3, out_channels = 64,
            kernel_size = 3, padding = 1,
            stride = 1, bias = False
        )
        self.bn1 = nn.BatchNorm2d(num_features = 64)
        
        self.resblock1 = ResNet_Block(
            num_inp_channels = 64, num_channels = 256,
            num_bottleneck_channels = 64, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = True
        )
        
        self.resblock2 = ResNet_Block(
            num_inp_channels = 256, num_channels = 256,
            num_bottleneck_channels = 64, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock3 = ResNet_Block(
            num_inp_channels = 256, num_channels = 256,
            num_bottleneck_channels = 64, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        # Downsample-
        self.resblock4 = ResNet_Block(
            num_inp_channels = 256, num_channels = 512,
            num_bottleneck_channels = 128, stride = 2,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock5 = ResNet_Block(
            num_inp_channels = 512, num_channels = 512,
            num_bottleneck_channels = 128, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock6 = ResNet_Block(
            num_inp_channels = 512, num_channels = 512,
            num_bottleneck_channels = 128, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock7 = ResNet_Block(
            num_inp_channels = 512, num_channels = 512,
            num_bottleneck_channels = 128, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        # Downsample-
        self.resblock8 = ResNet_Block(
            num_inp_channels = 512, num_channels = 1024,
            num_bottleneck_channels = 256, stride = 2,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock9 = ResNet_Block(
            num_inp_channels = 1024, num_channels = 1024,
            num_bottleneck_channels = 256, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock10 = ResNet_Block(
            num_inp_channels = 1024, num_channels = 1024,
            num_bottleneck_channels = 256, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock11 = resblock9 = ResNet_Block(
            num_inp_channels = 1024, num_channels = 1024,
            num_bottleneck_channels = 256, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock12 = resblock9 = ResNet_Block(
            num_inp_channels = 1024, num_channels = 1024,
            num_bottleneck_channels = 256, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock13 = resblock9 = ResNet_Block(
            num_inp_channels = 1024, num_channels = 1024,
            num_bottleneck_channels = 256, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        # Downsample-
        self.resblock14 = ResNet_Block(
            num_inp_channels = 1024, num_channels = 2048,
            num_bottleneck_channels = 512, stride = 2,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock15 = ResNet_Block(
            num_inp_channels = 2048, num_channels = 2048,
            num_bottleneck_channels = 512, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        self.resblock16 = ResNet_Block(
            num_inp_channels = 2048, num_channels = 2048,
            num_bottleneck_channels = 512, stride = 1,
            dropout = 0.2, use_1x1_conv = True,
            first_resblock = False
        )
        
        # self.avg_pool = nn.AvgPool2d(kernel_size = 3, stride = 2)
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size = (1, 1))
        self.output_layer = nn.Linear(in_features = 2048, out_features = 4)
        
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.resblock5(x)
        x = self.resblock6(x)
        x = self.resblock7(x)
        x = self.resblock8(x)
        x = self.resblock9(x)
        x = self.resblock10(x)
        x = self.resblock11(x)
        x = self.resblock12(x)
        x = self.resblock13(x)
        x = self.resblock14(x)
        x = self.resblock15(x)
        x = self.resblock16(x)
        x = self.avg_pool(x).squeeze()
        x = self.output_layer(x)
        return x
    
    
    def shape_computation(self, x):
        print(f"Input shape: {x.shape}")
        x = F.relu(self.bn1(self.conv1(x)))
        print(f"First conv layer output shape: {x.shape}")
        x = self.resblock1(x)
        print(f"ResBlock 1 output shape: {x.shape}")
        x = self.resblock2(x)
        print(f"ResBlock 2 output shape: {x.shape}")
        x = self.resblock3(x)
        print(f"ResBlock 3 output shape: {x.shape}")
        x = self.resblock4(x)
        print(f"ResBlock 4 output shape: {x.shape}")
        x = self.resblock5(x)
        print(f"ResBlock 5 output shape: {x.shape}")
        x = self.resblock6(x)
        print(f"ResBlock 6 output shape: {x.shape}")
        x = self.resblock7(x)
        print(f"ResBlock 7 output shape: {x.shape}")
        x = self.resblock8(x)
        print(f"ResBlock 8 output shape: {x.shape}")
        x = self.resblock9(x)
        print(f"ResBlock 9 output shape: {x.shape}")
        x = self.resblock10(x)
        print(f"ResBlock 10 output shape: {x.shape}")
        x = self.resblock11(x)
        print(f"ResBlock 11 output shape: {x.shape}")
        x = self.resblock12(x)
        print(f"ResBlock 12 output shape: {x.shape}")
        x = self.resblock13(x)
        print(f"ResBlock 13 output shape: {x.shape}")
        x = self.resblock14(x)
        print(f"ResBlock 14 output shape: {x.shape}")
        x = self.resblock15(x)
        print(f"ResBlock 15 output shape: {x.shape}")
        x = self.resblock16(x)
        print(f"ResBlock 16 output shape: {x.shape}")
        x = self.avg_pool(x).squeeze()
        print(f"Average pool output shape: {x.shape}")
        x = self.output_layer(x)
        print(f"Output layer shape: {x.shape}")
        del x
        return None
    

In [11]:
# Initialize ResNet-50 architecture-
model = ResNet50()

In [12]:
# Count number of layer-wise parameters and total parameters-
tot_params = 0
for param in model.parameters():
    # print(f"layer.shape = {param.shape} has {param.nelement()} parameters")
    tot_params += param.nelement()

print(f"Total number of trainable parameters in ResNet-50 CNN = {tot_params}")

Total number of trainable parameters in ResNet-50 CNN = 38080068


In [13]:
# Save random initial weights-
# torch.save(model.state_dict(), 'ResNet50_STL10_RotNet-Random_Weights.pth')

# Load randomly initialised weights-
model.load_state_dict(torch.load('ResNet50_STL10_RotNet-Random_Weights.pth'))

<All keys matched successfully>

In [14]:
# Use data parallelism, specify GPUs to use-
model = torch.nn.DataParallel(model, device_ids = [0, 1, 2, 3])

# Move to CUDA GPU-
model.to(0)

DataParallel(
  (module): ResNet50(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (resblock1): ResNet_Block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv4): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )

In [15]:
# Define loss function and optimizer-
cost_fn = nn.CrossEntropyLoss()

# optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum = 0.9, weight_decay = 5e-4)

In [16]:
# Decay lr at specified epochs by a factor of 10-
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer = optimizer, milestones = [30, 50, 70, 90],
    gamma = 0.1
)

In [17]:
def train_model_progress(model, train_loader, train_dataset):
    '''
    Function to perform one epoch of training by using 'train_loader'.
    Returns loss and number of correct predictions for this epoch.
    '''
    running_loss = 0.0
    running_corrects = 0.0
    
    # Place model to device-
    # model.to(device)
    # model.to(0)
    
    model.train()
    
    with tqdm(train_loader, unit = 'batch') as tepoch:
        for images, labels in tepoch:
            tepoch.set_description(f"Training: ")
            
            # Push to CUDA GPU-
            images = images.to(0)
            labels = labels.to(0)
            # images = images.to(device)
            # labels = labels.to(device)
            
            # Get model predictions-
            outputs = model(images)
            
            # Compute loss-
            J = cost_fn(outputs, labels)
            
            # Empty accumulated gradients-
            optimizer.zero_grad()
            
            # Perform backprop-
            J.backward()
            
            # Update parameters-
            optimizer.step()
            
            # global step
            # optimizer.param_groups[0]['lr'] = custom_lr_scheduler.get_lr(step)

            # step += 1
            
            # Compute model's performance statistics-
            running_loss += J.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            running_corrects += torch.sum(predicted == labels.data)
            
            tepoch.set_postfix(
                loss = running_loss / len(train_dataset),
                accuracy = (running_corrects.double().cpu().numpy() / len(train_dataset)) * 100
            )
            
    
    train_loss = running_loss / len(train_dataset)
    train_acc = (running_corrects.double() / len(train_dataset)) * 100
    

    # return running_loss, running_corrects
    return train_loss, train_acc.cpu().numpy()


In [ ]:
"""
def test_model_progress(model, test_loader, test_dataset):
    total = 0.0
    correct = 0.0
    running_loss_val = 0.0

    with torch.no_grad():
        with tqdm(test_loader, unit = 'batch') as tepoch:
            for images, labels in tepoch:
                tepoch.set_description(f"Validation: ")
                
                # Push to CUDA GPU-
                images = images.to(0)
                labels = labels.to(0)
                # images = images.to(device)
                # labels = labels.to(device)
                
                # Set model to evaluation mode-
                model.eval()
                
                # Place model to device-
                # model.to(device)
                # model.to(0)
            
                # Predict using trained model-
                outputs = model(images)
                _, y_pred = torch.max(outputs, 1)
                
                # Compute validation loss-
                J_val = cost_fn(outputs, labels)
                
                running_loss_val += J_val.item() * labels.size(0)
    
                # Total number of labels-
                total += labels.size(0)

                # Total number of correct predictions-
                correct += (y_pred == labels).sum()
                
                tepoch.set_postfix(
                    val_loss = running_loss_val / len(test_dataset),
                    val_acc = 100 * (correct.cpu().numpy() / total)
                )
            
        
    # return (running_loss_val, correct, total)
    val_loss = running_loss_val / len(test_dataset)
    val_acc = (correct / total) * 100

    return val_loss, val_acc.cpu().numpy()
"""

In [18]:
# Python3 dict to contain training metrics-
training_history = {}

# Initialize parameters saving 'best' models-
best_train_acc = 50

In [19]:
for epoch in range(1, num_epochs + 1):

    # Train and validate model for 1 epoch-
    train_loss, train_acc = train_model_progress(
        model = model, train_loader = unlabeled_loader,
        train_dataset = unlabeled_data
    )
    """
    val_loss, val_acc = test_model_progress(
        model = model, test_loader = test_loader,
        test_dataset = test_loader
    )
    """
    
    scheduler.step()
    current_lr = optimizer.param_groups[0]["lr"]
    
    print(f"Epoch: {epoch}; loss = {train_loss:.4f}, acc = {train_acc:.2f}%",
          # f" val loss = {val_loss:.4f}, val acc = {val_acc:.2f}%
          f" & LR = {current_lr:.8f}"
         )
    
    # Save training metrics to Python3 dict-
    training_history[epoch] = {
        'train_loss': train_loss, 'train_acc': train_acc,
        # 'val_loss': val_loss, 'val_acc': val_acc,
        'lr': current_lr
    }

    # Save best weights achieved until now-
    if (train_acc > best_train_acc):    
        # update 'best_val_loss' variable to lowest loss encountered so far-
        best_train_acc = train_acc

        print(f"Saving model with highest train acc = {train_acc:.2f}%\n")
        
        # Save trained model with 'best' validation accuracy-
        torch.save(model.state_dict(), "ResNet50_STL10_RotNet_best_acc.pth")
        

Training: : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [57:41<00:00,  2.21s/batch, accuracy=25.5, loss=1.48]


Epoch: 1; loss = 1.4784, acc = 25.51%  & LR = 0.10000000


Training: : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [59:17<00:00,  2.28s/batch, accuracy=31.5, loss=1.3]


Epoch: 2; loss = 1.3046, acc = 31.54%  & LR = 0.10000000


Training: : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [1:00:46<00:00,  2.33s/batch, accuracy=39.3, loss=1.18]


...

Epoch: 98; loss = 0.3730, acc = 86.19%  & LR = 0.00001000


Training: : 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [59:26<00:00,  2.28s/batch, accuracy=86.1, loss=0.373]


Epoch: 99; loss = 0.3732, acc = 86.11%  & LR = 0.00001000


Training: : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [1:00:37<00:00,  2.33s/batch, accuracy=86.3, loss=0.372]


Epoch: 100; loss = 0.3721, acc = 86.25%  & LR = 0.00001000
Saving model with highest train acc = 86.25%



In [21]:
torch.save(model.state_dict(), "ResNet50_STL10_RotNet_last_epoch.pth")

In [20]:
# Save training history as pickle file-
with open("STL10_ResNet50_Pretraining_history.pkl", "wb") as file:
    pickle.dump(training_history, file)